In [7]:
import tensorflow as tf
tf.__version__

'2.8.0'

## TFRecord
> - **HDFS**单块标准为64MB,如果要大于64x1.1,则会被分块.(为后期分布式做优化）
> - **TFRecord**官方建议是100mb-200mb,这里建议存为128mb或者192mb,取决于内存大小.

In [5]:
data_path = "/root/src/mooc/365data/star/48-data"

In [9]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset:
  print(element)


tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [12]:
dataset = tf.data.TextLineDataset(["1.txt", "2.txt"])

In [13]:
for i in dataset 

<TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [15]:
import tensorflow as tf
import numpy as np

image = np.random.rand(28, 28, 3)  # 一张图片的数据
label = 5  # 假设其 label 为 5

example = tf.train.Example(features=tf.train.Features(feature={
    # image 数据
    "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.tobytes()])),
    
    # 长、宽、通道数
    "height": tf.train.Feature(int64_list=tf.train.Int64List(value=[28])),
    "width": tf.train.Feature(int64_list=tf.train.Int64List(value=[28])),
    "channels": tf.train.Feature(int64_list=tf.train.Int64List(value=[3])),
    
    # label
    "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
}))

In [17]:
with tf.io.TFRecordWriter("./1.tfrecords") as f:
    f.write(example.SerializeToString())

In [24]:
def map_func(example):
    # feature 的属性解析表
    feature_map = {'image': tf.io.FixedLenFeature((), tf.string),
                   'height': tf.io.FixedLenFeature((), tf.int64),
                   'width': tf.io.FixedLenFeature((), tf.int64),
                   'channels': tf.io.FixedLenFeature((), tf.int64),
                   'label': tf.io.FixedLenFeature((), tf.int64)
                   }
    parsed_example = tf.io.parse_single_example(example, features=feature_map)

    # parsed_example["image"] 是 bytes 二进制数据，需要转化为 Tensor, 并告知转化后的 dtype
    # 我这里 np.random.rand 生成数据默认是 float64， 所以才这样写
    # 如果我们读取的图片数据是 0-255,则应该设置 out_type=tf.uint8
    image = tf.io.decode_raw(parsed_example["image"], out_type=tf.float64)
    height = parsed_example["height"]
    width = parsed_example["width"]
    channels = parsed_example["channels"]

    # 我们将数据转化为 bytes, 再转化为张量, 会转化为一个 1维数据
    # 这里提前保存 shape 信息，转化回来
    image = tf.reshape(image, [height, width, channels])
    label = parsed_example["label"]
    return image, label

In [25]:
dataset = tf.data.TFRecordDataset(["./1.tfrecords"])

In [26]:
dataset = dataset.map(map_func=map_func)

In [30]:
iterator = iter(dataset).next()

In [31]:
iterator

(<tf.Tensor: shape=(28, 28, 3), dtype=float64, numpy=
 array([[[0.01578101, 0.14111893, 0.09798259],
         [0.52016834, 0.86691399, 0.93896794],
         [0.73658048, 0.50462992, 0.72605521],
         ...,
         [0.73243633, 0.35001585, 0.13677089],
         [0.39685401, 0.50333596, 0.32483344],
         [0.21245795, 0.11349988, 0.7863852 ]],
 
        [[0.55600441, 0.22886614, 0.53126274],
         [0.96990155, 0.62389025, 0.19991873],
         [0.11611086, 0.98813112, 0.23262322],
         ...,
         [0.38316933, 0.34395524, 0.28818057],
         [0.01931731, 0.61110854, 0.40545598],
         [0.45612546, 0.04703484, 0.26684997]],
 
        [[0.8323555 , 0.41558551, 0.72045886],
         [0.48909087, 0.87690145, 0.66117589],
         [0.24592915, 0.68617509, 0.87539927],
         ...,
         [0.3851376 , 0.6285504 , 0.26700937],
         [0.13843931, 0.13151322, 0.36200444],
         [0.81706687, 0.40765513, 0.43464995]],
 
        ...,
 
        [[0.00174676, 0.45554259, 